# Nearest Neighbour Search

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

# Load the data
df_sales = pd.read_excel("../DATA CURRENT/SALES.xlsx")
df_parameters = pd.read_excel("../DATA CURRENT/L2-PARAMETERS.xlsx")

# Merge df_parameters into df_sales to get prices
df = df_parameters.copy()

# Define the columns to calculate stats for
columns_to_calculate = ['Price', 'Quantity']

# Create empty lists to store the results
quoted_price_min = []
quoted_price_max = []
quoted_price_avg = []
quoted_quantity_min = []
quoted_quantity_max = []
quoted_quantity_avg = []
ordered_price_min = []
ordered_price_max = []
ordered_price_avg = []
ordered_quantity_min = []
ordered_quantity_max = []
ordered_quantity_avg = []
average_price = []

# Loop through each Item Code in df_parameters
for item_code in df_parameters['Item Code']:
    # Filter df_sales for the current item code
    df_item_sales = df_sales[df_sales['Item Code'] == item_code]
    
    # Quoted prices and quantities
    df_quoted = df_item_sales[df_item_sales['Status'] == 'QUOTATO']
    quoted_price_min.append(df_quoted['Price'].min())
    quoted_price_max.append(df_quoted['Price'].max())
    quoted_price_avg.append(df_quoted['Price'].mean())
    quoted_quantity_min.append(df_quoted['Quantity'].min())
    quoted_quantity_max.append(df_quoted['Quantity'].max())
    quoted_quantity_avg.append(df_quoted['Quantity'].mean())
    
    # Ordered prices and quantities
    df_ordered = df_item_sales[df_item_sales['Status'] == 'ORDINATO']
    ordered_price_min.append(df_ordered['Price'].min())
    ordered_price_max.append(df_ordered['Price'].max())
    ordered_price_avg.append(df_ordered['Price'].mean())
    ordered_quantity_min.append(df_ordered['Quantity'].min())
    ordered_quantity_max.append(df_ordered['Quantity'].max())
    ordered_quantity_avg.append(df_ordered['Quantity'].mean())
    
    # Average price across both statuses
    avg_price = df_item_sales['Price'].mean(skipna=True)
    average_price.append(avg_price)

# Add the calculated stats to df
df['Quoted Price Min'] = quoted_price_min
df['Quoted Price Max'] = quoted_price_max
df['Quoted Price Avg'] = quoted_price_avg
df['Quoted Quantity Min'] = quoted_quantity_min
df['Quoted Quantity Max'] = quoted_quantity_max
df['Quoted Quantity Avg'] = quoted_quantity_avg
df['Ordered Price Min'] = ordered_price_min
df['Ordered Price Max'] = ordered_price_max
df['Ordered Price Avg'] = ordered_price_avg
df['Ordered Quantity Min'] = ordered_quantity_min
df['Ordered Quantity Max'] = ordered_quantity_max
df['Ordered Quantity Avg'] = ordered_quantity_avg
df['Average Price'] = average_price

# Save the resulting df to an Excel file
df.to_excel('L2 - MASTER_DATA.xlsx', index=False)

# Function to get user input dynamically
def get_user_input():
    layout = input("Does the product have a 'Threaded' Layout? (yes/no): ")
    sensing_element_options = df['Sensing Element'].unique()
    sensing_element = input(f"What type of Sensing Element is it? {tuple(sensing_element_options)}: ")
    case_material_options = df['Case Material'].unique()
    case_material = input(f"Enter Case Material {tuple(case_material_options)}: ")
    case_diameter = float(input("Enter Case Diameter: "))
    case_length = float(input("Enter Case Length: "))
    cable_material_options = df['Cable Material'].unique()
    cable_material = input(f"Enter Cable Material {tuple(cable_material_options)}: ")
    cable_length = float(input("Enter Cable Length: "))
    terminal_options = df['Terminal'].unique()
    terminal = input(f"Enter Terminal {tuple(terminal_options)}: ")
    cable_wires_number = int(input("Enter Cable Wires Number: "))

    return layout, sensing_element, case_material, case_diameter, case_length, cable_material, cable_length, terminal, cable_wires_number

# Get user input for filtering
layout, sensing_element, case_material, case_diameter, case_length, cable_material, cable_length, terminal, cable_wires_number = get_user_input()

# Convert user input to the required format
layout = "THREADED" if layout.lower() == "yes" else "Plain"
sensing_element = sensing_element.upper()

# Filter the dataframe based on user input
def filter_dataframe(df, layout, sensing_element):
    df_filtered = df[(df['Layout'] == layout) & (df['Sensing Element'] == sensing_element)]
    return df_filtered

df_filtered = filter_dataframe(df, layout, sensing_element)

# Check if the filtered DataFrame is empty
if df_filtered.empty:
    print("No records found for the given layout and sensing element.")
else:
    # Ensure 'Average Price' is in the filtered DataFrame
    if 'Average Price' not in df_filtered.columns:
        df_filtered['Average Price'] = df['Average Price']

    # Define relevant columns for each sensing element
    relevant_columns = {
        'NTC': ['R25', 'B25/85'],
        'PTC': ['Class'],
        'RTD': ['Material', 'R0', 'Class'],
        'MGO': ['TC Type'],
        'TCx': ['TC Type']
    }

    # Add common columns
    common_columns = ['Layout', 'Sensing Element', 'Case Material', 'Case Diameter', 'Case Length', 'Cable Material', 'Cable Length', 'Terminal', 'Cable Wires Number']
    if layout == "THREADED":
        common_columns.append('Case HEX')

    # Select only relevant columns for the chosen sensing element
    df_filtered = df_filtered[common_columns + relevant_columns.get(sensing_element, []) + ['Average Price']]

    # Function to get user input for nearest neighbor search
    def get_nn_user_input():
        user_input = {
            'Layout': layout,
            'Sensing Element': sensing_element,
            'Case Material': case_material,
            'Case Diameter': case_diameter,
            'Case Length': case_length,
            'Cable Material': cable_material,
            'Cable Length': cable_length,
            'Terminal': terminal,
            'Cable Wires Number': cable_wires_number
        }

        if layout == "THREADED":
            user_input['Case HEX'] = float(input("Enter Case HEX: "))

        if sensing_element == 'NTC':
            r25_options = df['R25'].dropna().unique()
            user_input['R25'] = input(f"Enter R25 {tuple(r25_options)}: ")
            user_input['B25/85'] = float(input("Enter B25/85: "))
        elif sensing_element == 'PTC':
            class_options = df['Class'].dropna().unique()
            user_input['Class'] = input(f"Enter Class {tuple(class_options)}: ")
        elif sensing_element == 'RTD':
            material_options = df['Material'].dropna().unique()
            r0_options = df['R0'].dropna().unique()
            class_options = df['Class'].dropna().unique()
            user_input['Material'] = input(f"Enter Material {tuple(material_options)}: ")
            user_input['R0'] = input(f"Enter R0 {tuple(r0_options)}: ")
            user_input['Class'] = input(f"Enter Class {tuple(class_options)}: ")
        elif sensing_element in ['MGO', 'TCx']:
            tc_type_options = df['TC Type'].dropna().unique()
            user_input['TC Type'] = input(f"Enter TC Type {tuple(tc_type_options)}: ")

        return user_input

    # Get user input
    user_input = get_nn_user_input()

    # Check for exact match
    exact_match = df_filtered[
        (df_filtered['Layout'] == user_input['Layout']) &
        (df_filtered['Sensing Element'] == user_input['Sensing Element']) &
        (df_filtered['Case Material'] == user_input['Case Material']) &
        (df_filtered['Case Diameter'] == user_input['Case Diameter']) &
        (df_filtered['Case Length'] == user_input['Case Length']) &
        (df_filtered['Cable Material'] == user_input['Cable Material']) &
        (df_filtered['Cable Length'] == user_input['Cable Length']) &
        (df_filtered['Terminal'] == user_input['Terminal']) &
        (df_filtered['Cable Wires Number'] == user_input['Cable Wires Number'])
    ]

    if layout == "THREADED":
        exact_match = exact_match[exact_match['Case HEX'] == user_input['Case HEX']]
    
    if sensing_element == 'NTC':
        exact_match = exact_match[(exact_match['R25'] == user_input['R25']) & (exact_match['B25/85'] == user_input['B25/85'])]
    elif sensing_element == 'PTC':
        exact_match = exact_match[exact_match['Class'] == user_input['Class']]
    elif sensing_element == 'RTD':
        exact_match = exact_match[(exact_match['Material'] == user_input['Material']) & (exact_match['R0'] == user_input['R0']) & (exact_match['Class'] == user_input['Class'])]
    elif sensing_element in ['MGO', 'TCx']:
        exact_match = exact_match[exact_match['TC Type'] == user_input['TC Type']]

    if not exact_match.empty:
        print("Exact match found:")
        print(exact_match)
    else:
        # Preprocess the data
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), [col for col in ['Cable Length', 'Case Diameter', 'Case Length', 'Case HEX'] if col in df_filtered.columns]),
                ('cat', OneHotEncoder(), [col for col in ['Layout', 'Sensing Element', 'Case Material', 'Cable Material', 'Terminal'] if col in df_filtered.columns])
            ]
        )

        # Generate interaction terms
        poly = PolynomialFeatures(interaction_only=True, include_bias=False)

        # Create pipeline for preprocessing, polynomial features, and regression
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('poly', poly),
            ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
        ])

        X_filtered = df_filtered.drop(columns=['Average Price'])
        y_filtered = df_filtered['Average Price']

        # Train the model for average_price
        pipeline.fit(X_filtered, y_filtered)
        feature_importance_price = pipeline.named_steps['regressor'].feature_importances_

        # Get the feature names after preprocessing and polynomial features
        one_hot_features = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out()
        feature_names = [col for col in ['Cable Length', 'Case Diameter', 'Case Length', 'Case HEX'] if col in df_filtered.columns] + list(one_hot_features)
        poly_features = poly.get_feature_names_out(feature_names)

        # Create a DataFrame for feature importance
        feature_importance_df = pd.DataFrame({
            'Feature': poly_features,
            'Importance': feature_importance_price
        })

        # Normalize the importance to get weights
        feature_importance_df['Weight'] = feature_importance_df['Importance'] / feature_importance_df['Importance'].sum()

        # Save as excel because too long
        feature_importance_df.to_excel('L2_feature_importance.xlsx', index=False)

        # Use the weights for the nearest neighbor model
        weights = feature_importance_df.set_index('Feature')['Weight'].to_dict()

        # Prepare the data for Nearest Neighbors using the calculated weights
        X_preprocessed = preprocessor.transform(X_filtered)  # Output is already a numpy array
        X_poly = poly.transform(X_preprocessed)

        # Apply weights to the preprocessed features
        weighted_features = np.array([weights[feature] for feature in poly_features])
        X_preprocessed_weighted = X_poly * weighted_features

        # Fit the NearestNeighbors model
        nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(X_preprocessed_weighted)

        # Convert user input to DataFrame
        input_df = pd.DataFrame([user_input])

        # Preprocess user input
        input_preprocessed = preprocessor.transform(input_df)  # Output is already a numpy array
        input_poly = poly.transform(input_preprocessed)
        input_preprocessed_weighted = input_poly * weighted_features

        # Find the nearest neighbors
        distances, indices = nbrs.kneighbors(input_preprocessed_weighted)

        # Normalize distances to compute similarity scores
        max_distance = np.max(distances)
        similarity_scores = 1 - (distances / max_distance)

        # Define target columns to include in the output
        target_columns = ['Item Code', 'Quoted Price Min', 'Quoted Price Max', 'Quoted Price Avg', 
                          'Quoted Quantity Min', 'Quoted Quantity Max', 'Quoted Quantity Avg',
                          'Ordered Price Min', 'Ordered Price Max', 'Ordered Price Avg',
                          'Ordered Quantity Min', 'Ordered Quantity Max', 'Ordered Quantity Avg']

        # Get the closest 5 products
        closest_products = df.iloc[indices[0]][target_columns].copy()
        closest_products['Similarity Score'] = similarity_scores[0]

        # Display the results
        print("Closest 5 products with similarity scores:")
        print(closest_products)

Does the product have a 'Threaded' Layout? (yes/no): no
What type of Sensing Element is it? ('NTC', 'PTC', 'RTD', 'TCx'): PTC
Enter Case Material ('TPE', 'Epoxy', 'Stainless Steel', 'Brass', 'Inconel', 'ABS', 'Copper'): Stainless Steel
Enter Case Diameter: 5
Enter Case Length: 15
Enter Cable Material ('TPE', 'PVC', 'SILICONE', 'TEFLON', 'VTX', 'TPEE'): TPE
Enter Cable Length: 150
Enter Terminal ('TINNED', 'CONNECTOR', 'TERMINALS', 'STRIPPED'): TINNED
Enter Cable Wires Number: 2
Enter Class ('121', '210', 'B', 'A', '221', '110', 'UNDEFINED', '1/3 DIN', '1/5 DIN'): 121
Closest 5 products with similarity scores:
         Item Code  Quoted Price Min  Quoted Price Max  Quoted Price Avg  \
0   103AT11/00150#              0.98              0.98              0.98   
63          AP0698              2.82              2.82              2.82   
52  AP0374/50/6X40               NaN               NaN               NaN   
12   103AT11/30/R1              1.09              1.09              1.09   
24 